In [28]:
from sys import path
from os.path import dirname as dir

In [29]:
path.append(dir(path[0]))

In [37]:
import yaml
from AgentLayer.RLAgents.A2C import A2C
from AgentLayer.Environment.PortfolioEnv import PortfolioEnv
from FinancialDataLayer.DataCollection.DataDownloader import DataDownloader
from FinancialDataLayer.DataProcessing.DefaultFeatureEngineer import DefaultFeatureEngineer
from AgentLayer.DataSplitter.TimeSeriesSplitter import TimeSeriesSplitter

In [31]:
# IMPORT .yaml FILE
# Gather user parameters
with open("..//user_params.yaml", "r") as stream:
    try:
        user_params = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

tickers = user_params["tickers"]
env_kwargs = user_params["env_params"]
train_params = user_params["train_params"]
policy_params = user_params["policy_params"]
test_params = user_params["test_params"]

In [32]:
# FETCH DATA
print("\nTest 3: Downloading from Yahoo.........")
downloaded_df = DataDownloader(start_date='2009-01-01',
                               end_date='2021-10-31',
                               ticker_list=tickers).download_from_yahoo()


Test 3: Downloading from Yahoo.........
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (12924, 8)


In [33]:
downloaded_df.head()

,date,open,high,low,close,volume,tic,day
0,2008-12-31,3.070357,3.133571,3.047857,2.606278,607541200,AAPL,2
1,2008-12-31,41.590000,43.049999,41.500000,32.005905,5443100,BA,2
2,2008-12-31,43.700001,45.099998,43.700001,30.628824,6277400,CAT,2
3,2008-12-31,72.900002,74.629997,72.900002,43.314426,9964300,CVX,2
4,2009-01-02,3.067143,3.251429,3.041429,2.771174,746015200,AAPL,4


In [34]:
# PREPROCESS DATA
print("\nTest 4: Feature engineer.........")

df_processed = DefaultFeatureEngineer(use_default=False,
                                      tech_indicator_list=env_kwargs["tech_indicator_list"],
                                      use_vix=True,
                                      use_turbulence=True,
                                      use_covar=True).extend_data(
    downloaded_df)  # included technical indicators as features



Test 4: Feature engineer.........
Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3231, 8)
Successfully added vix
Successfully added turbulence index
Successfully added covariances


In [35]:
df_processed.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,cov_list,return_list
0,2009-12-31,AAPL,7.611786,7.619643,7.520000,6.434926,352410800.0,3.0,0.105229,6.531584,5.633489,60.410526,155.827694,31.312031,6.105642,6.048183,21.680000,0.0,"[[0.0004566264425630261, 0.0002589232850643128...",tic AAPL BA CAT ...
0,2009-12-31,BA,55.000000,55.220001,54.049999,42.180122,2189400.0,3.0,0.448062,43.940991,41.779002,54.118718,17.483442,4.580979,42.211023,40.690113,21.680000,0.0,"[[0.0004566264425630261, 0.0002589232850643128...",tic AAPL BA CAT ...
0,2009-12-31,CAT,57.599998,57.959999,56.990002,40.802940,3859700.0,3.0,0.043858,42.300242,40.350236,51.872599,-74.367038,6.791854,41.526067,41.025310,21.680000,0.0,"[[0.0004566264425630261, 0.0002589232850643128...",tic AAPL BA CAT ...
0,2009-12-31,CVX,77.720001,77.779999,76.930000,46.806030,4246600.0,3.0,0.009378,47.587706,46.552031,52.439940,-69.788890,6.763381,47.276572,46.664710,21.680000,0.0,"[[0.0004566264425630261, 0.0002589232850643128...",tic AAPL BA CAT ...
1,2010-01-04,AAPL,7.622500,7.660714,7.585000,6.535086,493729600.0,0.0,0.119897,6.599507,5.619096,62.133681,168.777113,33.760635,6.113836,6.060276,20.040001,0.0,"[[0.0004430592837165872, 0.0002466259847550853...",tic AAPL BA CAT ...


In [38]:
# split data to train and test
splitter = TimeSeriesSplitter()
train = splitter.get_split_data(df_processed, '2009-01-01', '2020-06-30')
trade = splitter.get_split_data(df_processed, '2020-07-01', '2021-09-02')

In [39]:
# CREATE TRAIN ENV
env = PortfolioEnv(df=train, **env_kwargs)
env_train, _ = env.get_env()

In [40]:
# CREATE A2C AGENT
a2c = A2C(env=env_train, **policy_params["A2C_PARAMS"])

In [41]:
# TRAIN A2C AGENT
a2c.train_model(**train_params["A2C_PARAMS"])

In [43]:
# CREATE TEST ENV
env_test = PortfolioEnv(df=trade, **env_kwargs)

In [44]:
# TEST A2C AGENT
df_daily_return_a2c, df_actions_a2c = a2c.predict(environment=env_test, **test_params["A2C_PARAMS"])

            0
0    0.000000
1    0.005730
2    0.020702
3   -0.023303
4    0.009761
..        ...
291 -0.013793
292  0.013527
293  0.001444
294 -0.005088
295 -0.006350

[296 rows x 1 columns]
begin_total_asset:1000000
end_total_asset:1463791.608678601
Sharpe:  1.479957415324429
hit end!


In [45]:
a2c_weights = a2c.extract_weights(df_actions_a2c)

In [46]:
print("Portfolio weights per day:\n")
a2c_weights

Portfolio weights per day:



,date,weights
0,2020-07-01,tic weight 0 AAPL 0.25 1 BA 0.2...
1,2020-07-02,tic weight 0 AAPL 0.263374 1 BA 0...
2,2020-07-06,tic weight 0 AAPL 0.263244 1 BA 0...
3,2020-07-07,tic weight 0 AAPL 0.260862 1 BA 0...
4,2020-07-08,tic weight 0 AAPL 0.269979 1 BA 0...
...,...,...
291,2021-08-26,tic weight 0 AAPL 0.265650 1 BA 0...
292,2021-08-27,tic weight 0 AAPL 0.271801 1 BA 0...
293,2021-08-30,tic weight 0 AAPL 0.268928 1 BA 0...
294,2021-08-31,tic weight 0 AAPL 0.267420 1 BA 0...


In [47]:
# SAVE AGENT
a2c.save_model("../AgentLayer/RLAgents/a2c_model")

In [48]:
# LOAD AGENT
loaded_a2c_model = a2c.load_model("../AgentLayer/RLAgents/a2c_model")